In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Functions for bisection method
def bisection_method(fx, xmin_init, xmax_init, target):
    r""" Finding the closest solution, xopt, for the function f(x) = 0 by reducing the range between xmin and xmax 
    which corresponds to f(xmin) <= 0 and f(xmax) >= 0
    Parameters
    ----------
        fx: function
        xmin_init: int, float, np.ndarray
            Initial guess for xmin, make sure f(xmin) <= 0
        xmax_init: int, float, np.ndarray
            Initial guess for xmax, make sure f(xmax) >= 0
        target: float
            Operation breaks, when abs(f(xopt)) <= target
    Return
    ------
        xopt: float, np.ndarray
            Closest solution for f(x) = 0
    """
    xmin = xmin_init
    xmax = xmax_init
    print('Initial xmin = {}, xmax = {}'.format(xmin, xmax))
    print('fx(xmin) <= 0? {}'.format(fx(xmin) <= 0))
    print('fx(xmax) >= 0? {}'.format(fx(xmax) >= 0))
    if fx(xmin)*fx(xmax) > 0:
        raise AttributeError('Error in fx(xmin)*fx(xmax) > 0')
        
    # Initial error
    error = target + 1
    while np.abs(error) > target:
        xopt = (xmin + xmax)/2 # Calculate the middle point of xmax and xmin
        error = fx(xopt)
        print('Error = {}'.format(error))
        if np.abs(error) > target:
            # Swapping the either extreme with xopt to reduce the range
            if fx(xmin)* fx(xopt) > 0: 
                xmin = xopt
            else: 
                xmax = xopt
            print('Optimized xmin = {}, xmax = {}'.format(xmin, xmax))
        else:
            print('#======= Target reached!=======#')
            break
    return xopt


def flatten_npmat(vec_like):
    vec = vec_like.flatten()
    vec = np.array(vec.T)
    return np.reshape(vec, vec.shape[0])


In [3]:
# Parameter setting
M = 4 # number of TX elements
K = 3 # number of RX
lam = 0
eta = 10**-2 # Power limit for the interference 
sigma = 1.0
snr = 0 # dB
pmax = (sigma**2)* 10**(snr/10)
pmax

1.0

In [4]:
# Channel
np.random.seed(0)
H = np.matrix(1/np.sqrt(2)*(np.random.random((M, K)) + 1j*np.random.random((M, K))))
# Mu: intial setting
Mu = 1*np.ones((K, K)) #np.random.random((K, K)) #
Mu = Mu - Mu.diagonal()*np.identity(K)
print(H)

[[0.38806975+0.40166816j 0.50571525+0.65449566j 0.42621807+0.05023008j]
 [0.38529059+0.06160972j 0.29956918+0.01429657j 0.45671611+0.58875114j]
 [0.30942088+0.55023992j 0.63057874+0.61519149j 0.68141247+0.69198767j]
 [0.2711341 +0.56509044j 0.55983414+0.32631519j 0.37398518+0.55191747j]]


In [5]:
# Weight vectors
def weight_vector(h_k, mat):
    mat_inv = np.linalg.inv(mat)
    return np.dot(mat_inv, h_k)

def scaling_matrix(H, k, lam, method, mu_k = None):
    M = H.shape[0] # Number of TX elements
    K = H.shape[1] # Number of RX
    mat = np.matrix((0 + 0j)* np.ones((M, M)))
    mat += lam* np.identity(M)
    for j in range(K):
        if j != k:
            h_j = H[:, j]
            if method == 1:
                mat += np.dot(h_j, h_j.H)
            elif method == 2:
                mat += mu_k[j]* np.dot(h_j, h_j.H)
    return mat


In [6]:
k = 0
h_k = H[:, k]
lam = 1
# method 1
A0_1 = scaling_matrix(H, k, lam, 1, Mu[:, k])
v0_1 = weight_vector(h_k, A0_1)
print(A0_1)
print(v0_1)
# method 2
A0_2 = scaling_matrix(H, k, lam, 2, Mu[:, k])
v0_2 = weight_vector(h_k, A0_2)
print(A0_2)
print(v0_2)

[[1.86829739+0.j         0.38508742-0.03915875j 1.04672235-0.15911092j
  0.68381064-0.01506545j]
 [0.38508742+0.03915875j 1.64516359+0.j         0.91631766-0.09013684j
  0.66812134-0.12163566j]
 [1.04672235+0.15911092j 0.91631766+0.09013684j 2.71926   +0.j
  1.19052409+0.02134747j]
 [0.68381064+0.01506545j 0.66812134+0.12163566j 1.19052409-0.02134747j
  1.86437368+0.j        ]]
[[ 0.16563278+0.10914029j]
 [ 0.174334  -0.13756325j]
 [-0.01487277+0.08318266j]
 [ 0.02265569+0.24636643j]]
[[1.86829739+0.j         0.38508742-0.03915875j 1.04672235-0.15911092j
  0.68381064-0.01506545j]
 [0.38508742+0.03915875j 1.64516359+0.j         0.91631766-0.09013684j
  0.66812134-0.12163566j]
 [1.04672235+0.15911092j 0.91631766+0.09013684j 2.71926   +0.j
  1.19052409+0.02134747j]
 [0.68381064+0.01506545j 0.66812134+0.12163566j 1.19052409-0.02134747j
  1.86437368+0.j        ]]
[[ 0.16563278+0.10914029j]
 [ 0.174334  -0.13756325j]
 [-0.01487277+0.08318266j]
 [ 0.02265569+0.24636643j]]


In [7]:
# Base to store the obtained results
V = np.matrix((0 + 0j)* np.ones((M, K)))
P = np.zeros((K, K))# Power
P_v = np.zeros(K)
sinr = np.zeros(K)
# Targets
target_pk = 0.0001*pmax/K
target_pj = 0.001*eta
# Initial setting
# lambda: larger lambda -> smaller norm(v_k)
lam_min = 10
lam_max = 10**(-8)#10**(-20)
mu_kj_min = 10**6 #-1.9
mu_kj_max = 10**(-6) #0
lam_opt = np.zeros(K)
Mu_opt = np.copy(Mu)

for k in range(Mu.shape[1]): 
    print('======================')
    print('RX No. {}'.format(k))
    print('======================')
    satisfied = False
    h_k = H[:, k]
    
    while satisfied == False:
        # Power constraint
        def flambda(lam):
            mat_k = scaling_matrix(H, k, lam, 2, Mu_opt[:, k])
            v_k = weight_vector(h_k, mat_k)
            power_k = np.linalg.norm(v_k)**2
            return power_k - pmax/K

        lam = bisection_method(flambda, lam_min, lam_max, target_pk) 
        lam_opt[k] = lam
        print('***lam_opt: {}'.format(lam_opt))

        # Interference constraint
        for j in range(Mu.shape[1]):
            if j != k:
                h_j = H[:, j]
                mu_k = np.copy(Mu_opt[:, k])
                mat_k = scaling_matrix(H, k, lam_opt[k], 2, mu_k)
                v_k = weight_vector(h_k, mat_k)
                interference = np.abs(np.dot(h_j.H, v_k))**2
                print('Interference: {}'.format(interference))
                
                if interference > eta:
                    def fmu(mu_kj):
                        mu_k[j] = mu_kj 
                        mat_k = scaling_matrix(H, k, lam_opt[k], 2, mu_k)
                        v_k = weight_vector(h_k, mat_k)
                        interference = np.abs(np.dot(h_j.H, v_k))**2
                        return interference - eta
                    
                    mu_kj_opt = bisection_method(fmu, mu_kj_min, mu_kj_max, target_pj) 
                    print('***mu_kj_opt : {}'.format(mu_kj_opt))
                    Mu_opt[j, k] = mu_kj_opt 

        # Check if the constraints are still fulfilled
        mat_k = scaling_matrix(H, k, lam_opt[k], 2, Mu_opt[:, k])
        v_k = weight_vector(h_k, mat_k)
        V[:, k] = v_k
        P_v[k] = np.linalg.norm(v_k)**2
        # Signal/interference power
        p_k = np.dot(H.H, v_k) # size = (K, 1)
        p_k = flatten_npmat(p_k) # size = K
        p_k = np.abs(p_k)**2
        P[:, k] = p_k # Store the results regarding the power
        p_int = p_k # Interference power
        p_int[k] = 0.0 # "Interference" of the desired signal = 0 
        print('Maximal interference power: {}'.format(max(p_int)))
        if np.abs(P_v[k] - pmax/K) > target_pk or max(p_int) - eta > target_pj:
            satisfied = False
            print('!!! Not yet satisfied !!!')
        else:
            satisfied = True
            print('!!! Satified :) !!!')
            p_all = np.sum(p_k)
            sinr[k] = P[k, k] / (p_all - P[k, k] + sigma**2)
           
        
print('Mu_opt: {}'.format(Mu_opt))
print('lam_opt: {}'.format(lam_opt))

RX No. 0
Initial xmin = 10, xmax = 1e-08
fx(xmin) <= 0? True
fx(xmax) >= 0? True
Error = -0.31368563648519254
Optimized xmin = 5.000000005, xmax = 1e-08
Error = -0.2857783325289466
Optimized xmin = 2.5000000075, xmax = 1e-08
Error = -0.21819709964131268
Optimized xmin = 1.25000000875, xmax = 1e-08
Error = -0.012607117701413928
Optimized xmin = 0.625000009375, xmax = 1e-08
Error = 0.7244212824380474
Optimized xmin = 0.625000009375, xmax = 0.3125000096875
Error = 0.1836677109161184
Optimized xmin = 0.625000009375, xmax = 0.46875000953125
Error = 0.06554836372496847
Optimized xmin = 0.625000009375, xmax = 0.546875009453125
Error = 0.02276015267215259
Optimized xmin = 0.625000009375, xmax = 0.5859375094140624
Error = 0.004265377907390955
Optimized xmin = 0.625000009375, xmax = 0.6054687593945312
Error = -0.004361081343531581
Optimized xmin = 0.6152343843847656, xmax = 0.6054687593945312
Error = -9.689854345945603e-05
Optimized xmin = 0.6103515718896484, xmax = 0.6054687593945312
Error = 0.

Error = [[-0.00997855]]
Optimized xmin = 61.03515724993897, xmax = 1e-06
Error = [[-0.00991596]]
Optimized xmin = 30.517579124969483, xmax = 1e-06
Error = [[-0.00967735]]
Optimized xmin = 15.258790062484742, xmax = 1e-06
Error = [[-0.00880813]]
Optimized xmin = 7.629395531242371, xmax = 1e-06
Error = [[-0.00589761]]
Optimized xmin = 3.8146982656211854, xmax = 1e-06
Error = [[0.00252213]]
Optimized xmin = 3.8146982656211854, xmax = 1.9073496328105928
Error = [[-0.00336279]]
Optimized xmin = 2.8610239492158893, xmax = 1.9073496328105928
Error = [[-0.00110923]]
Optimized xmin = 2.3841867910132413, xmax = 1.9073496328105928
Error = [[0.00047439]]
Optimized xmin = 2.3841867910132413, xmax = 2.145768211911917
Error = [[-0.00036604]]
Optimized xmin = 2.264977501462579, xmax = 2.145768211911917
Error = [[4.10000167e-05]]
Optimized xmin = 2.264977501462579, xmax = 2.205372856687248
Error = [[-0.00016568]]
Optimized xmin = 2.2351751790749135, xmax = 2.205372856687248
Error = [[-6.31447758e-05]]


Error = [[0.00011696]]
Optimized xmin = 10.251999901356935, xmax = 10.132790611806273
Error = [[1.70230446e-05]]
Optimized xmin = 10.251999901356935, xmax = 10.192395256581605
Error = [[-3.23933899e-05]]
Optimized xmin = 10.22219757896927, xmax = 10.192395256581605
Error = [[-7.73099508e-06]]
#======= Target reached!=======#
***mu_kj_opt : 10.207296417775439
Interference: [[0.00466237]]
Maximal interference power: 0.009992269004919683
!!! Not yet satisfied !!!
Initial xmin = 10, xmax = 1e-08
fx(xmin) <= 0? True
fx(xmax) >= 0? True
Error = -0.319854395997341
Optimized xmin = 5.000000005, xmax = 1e-08
Error = -0.29226087192678596
Optimized xmin = 2.5000000075, xmax = 1e-08
Error = -0.18901506515852276
Optimized xmin = 1.25000000875, xmax = 1e-08
Error = 0.21083493372515733
Optimized xmin = 1.25000000875, xmax = 0.625000009375
Error = -0.08429573867262227
Optimized xmin = 0.9375000090625001, xmax = 0.625000009375
Error = 0.020067931030766983
Optimized xmin = 0.9375000090625001, xmax = 0.7

Optimized xmin = 3906.250000996094, xmax = 1e-06
Error = [[-0.00999965]]
Optimized xmin = 1953.125000998047, xmax = 1e-06
Error = [[-0.00999861]]
Optimized xmin = 976.5625009990235, xmax = 1e-06
Error = [[-0.00999448]]
Optimized xmin = 488.28125099951177, xmax = 1e-06
Error = [[-0.00997814]]
Optimized xmin = 244.14062599975588, xmax = 1e-06
Error = [[-0.00991411]]
Optimized xmin = 122.07031349987794, xmax = 1e-06
Error = [[-0.00966843]]
Optimized xmin = 61.03515724993897, xmax = 1e-06
Error = [[-0.00876244]]
Optimized xmin = 30.517579124969483, xmax = 1e-06
Error = [[-0.00566053]]
Optimized xmin = 15.258790062484742, xmax = 1e-06
Error = [[0.00365553]]
Optimized xmin = 15.258790062484742, xmax = 7.629395531242371
Error = [[-0.00290131]]
Optimized xmin = 11.444092796863556, xmax = 7.629395531242371
Error = [[-0.00041313]]
Optimized xmin = 9.536744164052964, xmax = 7.629395531242371
Error = [[0.00135273]]
Optimized xmin = 9.536744164052964, xmax = 8.583069847647668
Error = [[0.00041389]]

In [8]:
print('Mu_opt: {}'.format(Mu_opt))
print('lam_opt: {}'.format(lam_opt))

Mu_opt: [[ 0.         10.7139359   9.290875  ]
 [ 5.87850909  0.          1.        ]
 [ 2.21096079  1.          0.        ]]
lam_opt: [0.52711488 0.80375672 1.2854004 ]


In [9]:
print('Power: \n {}'.format(P))
print('SINR: {}dB'.format(10*(np.log10(sinr))))

Power: 
 [[0.06226368 0.00999913 0.01000239]
 [0.00879448 0.14353156 0.00784   ]
 [0.01000194 0.00378955 0.28007424]]
SINR: [-11.86465061  -7.82700165  -4.2064674 ]dB


In [10]:
print(P_v)

[0.3333516  0.33335468 0.33331987]
